## Data description

### explanation of data (file it came from, which response is being predicted, which sequences)
The DNA sequences that we used are from "concatenated.fasta". This file is generated as explained 
[here](https://github.com/solislemuslab/dna-nn/blob/master/notebook.md) with OG_X.aligned.fasta files, one per OG.

The result/response file that we use used is Perron_predictAMR_2021_updated.xlsx, received in Dec. 6th, 2021 by Email. We used all 8 parameters for classification and binarized the data into 2 class (less susceptible or more susceptible) with data in tab "binarized succeptibility key". Among them, only classifications on the measurements delta.carb.max.od1 and delta.toby.max.rate works with testing accuracy around 75%.

We processed the new response with this [jupyter notebook](https://github.com/solislemuslab/dna-nn/blob/master/nn-methods/yuren/new-data-process.ipynb) to csv files before training models on CHTC server.


### data encoding (how are sequences encoded)

The data are first encoded into DNA words (3 nucleotides as one unit), which is done before in DNA-NN-Theory as inspired by this [paper](http://dx.doi.org/10.4236/jbise.2016.95021). For examnple, the input sequence 'ATGCA', will be eoncoded into words 'ATG', 'TGC', 'GCA' and each word will be represented by an integer. This will encode the DNA sequences with length N into the word sequence with length N-2. Then, we use one-hot encoding on the resulting word sequence the same as [DNA-NN-Theory paper](https://arxiv.org/abs/2012.05995).



### data splitting

We randomly splitted the data to use 70% of dataset for training, 15% for validating, and 15% for testing.


### data augmentation and simulation

We augmented the data by simulating sequences with iqtree2 from concatenated.fasta with commend "iqtree2 --alisim alignment_mimic -s path-to-concatenated.fasta." This will doulbe our sample size. Then, we augmented the data with reverse complement, which is inspired by this [paper](https://doi.org/10.1016/j.ab.2021.114120), to double the sample size again.

Besides, we tried not using data augmentation or bootstraping as inspired by this [paper](https://www.pnas.org/content/93/23/13429). However, this does not work for our model. (the training accuracy did not increase at all, which means that the model was not albe to be trained).

## Model characteristics

### NN architecture

We used the same model cnn_nguyen_conv1d_2_conv2d as the paper for DNA-NN-Theory and [github page](https://github.com/solislemuslab/dna-nn-theory/blob/master/cnn/dna_nn/model.py#L23).

### when was learning stopped? Early stopped or fixed number of epochs

During the training process, we set the number of training epoches to 300. We also set earlystopping with patience as 50 epoches on validation loss so that the process will end with 50 epoches without reduction on validation loss (reducing validation loss means that the model is still learning from the training data and valudation results are closer to the expected output. At the end of each epoch, we validate the model and save the model if the validation loss is lower than those of all the epoches before so that we only save the best model. The model usually stop after trainig for around 100 epoches.

### Optimizer used, tuning parameters

We used the adam optimizer with binary crossentropy as loss, and default learning rate (0.001).

## Results

The models were trained on CHTC servers with the scripts [here](https://github.com/solislemuslab/dna-nn/tree/master/nn-methods/yuren/CHTC-scripts/cnn)

### Validation/testing accuracy

Among all 8 measures, only delta.toby.max.rate and delta.carb.max.od produced reasonable results. When using other methods, the training accuracy remain around 50% during the training process, which means that the model is not able to be trained. We note that even for delta.toby.max.rate and delta.carb.max.od, the model suffer from over-fitting as the training accuracy keep increasing to around 98% - 100% while the validation accuracy does not (which means that the model is fitting the training data too much but not generalized for data not in the training set).

Metrix | delta.toby.max.rate - testing | delta.toby.max.rate - validation | delta.carb.max.od - testing | delta.carb.max.od - validation |
| --- | --- | --- | --- | --- |
Accuracy | 0.7333333492279053 | 0.7096773982048035 | 0.774193525314331 | 0.7419354915618896 |
ROC Score | 0.7288888888888889 | 0.7541666666666667 | 0.8583333333333334 | 0.7833333333333332 |
Precision | 0.6666666666666666 | 0.7142857142857143 | 0.8 | 0.8333333333333334 |
Recall |  0.9333333333333333 | 0.6666666666666666 | 0.75 | 0.625 |
F1 score | 0.7777777777777778 | 0.689655172413793 | 0.7741935483870969 | 0.7142857142857143 |

### Metrics calculation

The raw results were output from CHTC with the scripts [here](https://github.com/solislemuslab/dna-nn/tree/master/nn-methods/yuren/CHTC-scripts/test)

In [28]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

def print_metrics(line):
  """
  calculate roc, precision, recall, and f1 and print results
  """
  splited = line.split(", ")
  types = [0 if x == "false" else 1 for x in splited[0].split("; ")]
  accs = float(splited[2])
  y_score = [float(x) for x in splited[3].split(";")]
  y_true = [0 if x == "false" else 1 for x in splited[4].split(";")] # true classes
  y_predict = [0 if x < 0.5 else 1 for x in y_score] # predicted classes

  roc = roc_auc_score(y_true, y_score)
  precision = precision_score(y_true, y_predict)
  recall = recall_score(y_true, y_predict)
  f1 = f1_score(y_true, y_predict)
  
  print("Accuracy:", accs)
  print("ROC score:", roc)
  print("Precision:",precision)
  print("Recall:", recall)
  print("F1 score:", f1)

#### delta.toby.max.rate

In [29]:
# delta.toby.max.rate
# Testing
test_line = "false; true, 0.5862652063369751, 0.7333333492279053, 0.554656;0.7812401;0.6250616;0.7381644;0.6881355;0.7485109;0.63956875;0.78092486;0.6905692;0.627747;0.5902186;0.57867014;0.6876658;0.34502226;0.5788807;0.8172598;0.14883386;0.70224726;0.27760255;0.18484908;0.33793905;0.2681906;0.57557243;0.18064763;0.71707284;0.28378478;0.5720832;0.5967484;0.2693308;0.81272405, true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;false;false;false;false;false;false;false;false;false;false;false;false;false;false;false"

print_metrics(test_line)

"""
Accuracy: 0.7333333492279053
ROC score: 0.7288888888888889
Precision: 0.6666666666666666
Recall: 0.9333333333333333
F1 score: 0.7777777777777778
"""

Accuracy: 0.7333333492279053
ROC score: 0.7288888888888889
Precision: 0.6666666666666666
Recall: 0.9333333333333333
F1 score: 0.7777777777777778


In [30]:
# delta.toby.max.rate
# Validation
val_line = "false; true, 0.609246551990509, 0.7096773982048035, 0.689087;0.7348399;0.5383368;0.66500074;0.4285111;0.08421713;0.3120905;0.8420089;0.55674934;0.75452745;0.23076646;0.3623125;0.61170137;0.5390358;0.5371771;0.4033272;0.04389373;0.49975514;0.33886668;0.52748644;0.6639347;0.22594659;0.6631823;0.13013911;0.019412395;0.16971086;0.04294053;0.064686224;0.61315256;0.4866849;0.40328816, true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;false;false;false;false;false;false;false;false;false;false;false;false;false;false;false;false"

print_metrics(val_line)

"""
Accuracy: 0.7096773982048035
ROC score: 0.7541666666666667
Precision: 0.7142857142857143
Recall: 0.6666666666666666
F1 score: 0.689655172413793
"""

Accuracy: 0.7096773982048035
ROC score: 0.7541666666666667
Precision: 0.7142857142857143
Recall: 0.6666666666666666
F1 score: 0.689655172413793


#### delta.carb.max.od

In [38]:
# delta.carb.max.od
# Testing
test_line = "false; true, 0.47950658202171326, 0.774193525314331, 0.7764048;0.64018786;0.8513884;0.87898093;0.9226941;0.33160338;0.88719803;0.96011883;0.24542268;0.75561845;0.9037272;0.95036954;0.95133734;0.7330078;0.21270643;0.064618275;0.10576571;0.67048174;0.13789867;0.1108837;0.2284823;0.14255854;0.583565;0.35511088;0.22413185;0.046967346;0.18738188;0.060229406;0.20567626;0.8247885;0.24959514, true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;false;false;false;false;false;false;false;false;false;false;false;false;false;false;false"

print_metrics(test_line)


'''
Accuracy: 0.774193525314331
ROC score: 0.8583333333333334
Precision: 0.8
Recall: 0.75
F1 score: 0.7741935483870969
'''

Accuracy: 0.774193525314331
ROC score: 0.8583333333333334
Precision: 0.8
Recall: 0.75
F1 score: 0.7741935483870969


In [36]:
# delta.carb.max.od
# Validation
val_line = "false; true, 0.6129928231239319, 0.7419354915618896, 0.87241066;0.89877737;0.11440517;0.27015358;0.9750905;0.92374074;0.84539783;0.86430043;0.20370886;0.055812284;0.8570183;0.9013735;0.9276164;0.1199808;0.86320204;0.22311008;0.45386583;0.0791924;0.889917;0.072813615;0.20723993;0.6823132;0.16593266;0.2191381;0.10064242;0.10897443;0.09760747;0.33146143;0.11017249;0.045579046;0.4616803, true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;true;false;false;false;false;false;false;false;false;false;false;false;false;false;false;false"

print_metrics(val_line)

"""
Accuracy: 0.7419354915618896
ROC score: 0.7833333333333332
Precision: 0.8333333333333334
Recall: 0.625
F1 score: 0.7142857142857143
"""

Accuracy: 0.7419354915618896
ROC score: 0.7833333333333332
Precision: 0.8333333333333334
Recall: 0.625
F1 score: 0.7142857142857143
